In [1]:
pip install pandas

   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.3/11.6 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.6 MB 2.0 MB/s eta 0:00:06
   -- ------------------------------------- 0.8/11.6 MB 2.0 MB/s eta 0:00:06
   --- ------------------------------------ 1.0/11.6 MB 1.3 MB/s eta 0:00:09
   --- ------------------------------------ 1.0/11.6 MB 1.3 MB/s eta 0:00:09
   ---- ----------------------------------- 1.3/11.6 MB 1.1 MB/s eta 0:00:10
   ---- ----------------------------------- 1.3/11.6 MB 1.1 MB/s eta 0:00:10
   ----- ---------------------------------- 1.6/11.6 MB 902.1 kB/s eta 0:00:12
   ----- ---------------------------------- 1.6/11.6 MB 902.1 kB/s eta 0:00:12
   ------ --------------------------------- 1.8/11.6 MB 838.9 kB/s eta 0:00:12
   ------ --------------------------------- 1.8/11.6 MB 838.9 kB/s eta 0:00:12
   ------- -

# Exploring the Dataset 

In [5]:
import pandas as pd

In [6]:
data = pd.read_csv("Amharic News Dataset.csv")

In [7]:
print(data.head())  # First 5 rows
print(data.tail())  # Last 5 rows


                                      headline category               date  \
0  የኦሊምፒክ ማጣሪያ ተሳታፊዎች የሚለዩበት ቻምፒዮና እየተካሄደ ይገኛል     ስፖርት   January 14, 2021   
1                                   አዲስ ዘመን ድሮ     መዝናኛ  December 28, 2020   
2             የአረንጓዴ ጎርፍ በጎ አድራጎት አምባሳደሮች ተሰየሙ     ስፖርት    January 6, 2021   
3        የሊጉ በቢዝነስ ሞዴል መመራት አበረታች ጅምር መሆኑ ተገለጸ     ስፖርት    January 6, 2021   
4    የኦሊምፒክ ሥራ አስፈፃሚው እስከ ቶኪዮ ኦሊምፒክ ማግስት ይቀጥላል     ስፖርት    January 6, 2021   

  views                                            article  \
0     2  ብርሃን ፈይሳየኢትዮጵያ ቦክስ ፌዴሬሽን በየዓመቱ የሚያዘጋጀው የክለቦች ቻ...   
1     4   የአዲስ ዘመን ጋዜጣ ቀደምት ዘገባዎች በእጅጉ ተነባቢ ዛሬም ላገኛቸው በ...   
2     6  ቦጋለ አበበየአዲስ አበባ ከተማ አስተዳደር ስፖርት ኮሚሽን ከኢትዮጵያ አረ...   
3     5  ብርሃን ፈይሳአዲስ አበባ፡- የኢትዮጵያ ፕሪምየር ሊግ በሼር ካምፓኒ እንዲ...   
4    12  ቦጋለ አበበ የኢትዮጵያ ኦሊምፒክ ኮሚቴ አርባ አምስተኛ መደበኛ ጠቅላላ ጉ...   

                                link  
0  https://www.press.et/Ama/?p=39481  
1  https://www.press.et/Ama/?p=38334  
2  https://www.press.et/Ama/?p=39010  
3 

In [8]:
print(data.info())  # Data types, non-null values, memory usage


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51483 entries, 0 to 51482
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   headline  51470 non-null  object
 1   category  51482 non-null  object
 2   date      51483 non-null  object
 3   views     51483 non-null  object
 4   article   51483 non-null  object
 5   link      51483 non-null  object
dtypes: object(6)
memory usage: 2.4+ MB
None


In [9]:
print(data.shape)  # (rows, columns)

(51483, 6)


In [11]:
print(data.isnull().sum())  # Count missing values in each column

headline    13
category     1
date         0
views        0
article      0
link         0
dtype: int64


In [ ]:
data=data.dropna() # removing rows with null values because they are not many
print(data.shape)  # (rows, columns)

(51470, 6)
